# Notes:
- Split the sources into groups
- 200 sources per group
- 40 groups
- 20hr compute requested per group

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import scipy 
from scipy.interpolate import RegularGridInterpolator

%matplotlib inline
import matplotlib.pyplot as plt

from pyfrac.utils.sofiutils import *

# FILE PATHS

In [2]:
# PANCAKE ONES 
shaheen_cp_loc = '/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/UtahFORGE_GriddedRecs_CroppedModel'
source_paths = os.path.join(shaheen_cp_loc,'inputs/multishot_sources')
print(source_paths)

# 'FAKE' Shaheen Path
# shaheen_proj_loc = '/scratch/birniece/SOFI_UtahFORGE/explosive_multishot/UtahFORGE_GriddedRecs_CroppedModel'
shaheen_proj_loc = '/home/birniece/SOFI_UtahFORGE/explosive_multishot/UtahFORGE_GriddedRecs_CroppedModel'

rec_file = os.path.join(shaheen_proj_loc,'inputs/UtahFORGE_griddedrecsFS_xzy_2m.dat')
model_dir = os.path.join(shaheen_proj_loc,'inputs/model/UtahFORGE_CroppedCentralVolume_XYZ_2m_xyz')

/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/UtahFORGE_GriddedRecs_CroppedModel/inputs/multishot_sources


In [3]:
sofi_src_dir = os.path.join(shaheen_proj_loc,'inputs/multishot_sources')

# sofimaster = '/project/k1056/SOFI_FDmodelling/sofi3d/bin/sofi3D'
sofimaster = '/home/birniece/sofi3d/bin/sofi3D'

### SLURM FILE CREATION FUNC

In [4]:
def slurmfilecreation(slurmfile, gid, sofimaster, jsonloc, joutloc, write=True):
    slurmstr='''#!/bin/bash -l
#SBATCH --job-name=UTAH_multishot_group%.2i
#SBATCH --nodes=32
#SBATCH --partition=workq
#SBATCH --time=23:00:00
#SBATCH --error=sofi.%%J.err 
#SBATCH --output=sofi.%%J.out

#OpenMP settings:
export OMP_NUM_THREADS=1

# RUN MODELLING JOB
srun --hint=nomultithread --ntasks=1000 %s %s.json > %s.jout
'''%(gid, sofimaster, jsonloc, joutloc)
    
    if write:
        text_file = open(slurmfile, "wt")
        text_file.writelines(slurmstr)
        text_file.close()

# MODELLING PARAMS (Indep. of src locs)

### MODEL PARAMETERS - Get these from the script that made the models

In [5]:
# TIME PARAMETERS
dt = 0.0001
tdur = 1.0

In [6]:
# SPATIAL PARAMETERS
dx = dy = dz = 2.
n_xzy = (500, 1000, 500)
# n_xzy = (500, 500, 920)


# dx = dy = dz = 4.
# n_xzy = (440, 500, 480)
# xmax = 1720
# ymax = 1920# y is y, I fix the SOFI ordering later
# zmax = 1680

# nx_homo = int(xmax//dx)
# ny_homo = int(ymax//dy)
# nz_homo = int(zmax//dz)
# n_xzy = [nx_homo, nz_homo, ny_homo]

d_xzy = [dx, dz, dy]

print(n_xzy[0]*dx,n_xzy[1]*dx,n_xzy[2]*dx)

1000.0 2000.0 1000.0


In [7]:
# BOUNDARIES & FREE SURFACE
nbounds = 49
free_surface = True

In [8]:
# SNAP PARAMETERS
snap_start = dt
snap_end = tdur
snap_step = 100*dt
tsnap_params = [snap_start, snap_end, snap_step]
snap_sbsmp_xyz = 1

# SEISMOGRAM PARAMETERS
smgrm_ndt = 10

In [9]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings(rec_file=rec_file)

# MODELLING
t_str = get_time_str(dt, tdur)
mod_strs = get_subsurfmod_str(n_xzy, 
                              d_xzy,
                              ' ',
                              moddir=model_dir,
                              np_xzy=[10,10,10]
                             )
bndry_str = get_boundary_str(fs=free_surface, 
                             npad=nbounds, 
                             cpml=True)


# SOURCES

In [10]:
xmid = n_xzy[0]*dx/2  # Place Central
ymid = n_xzy[2]*dx/2  # Place Central
zloc = 1500

nxs = 30
nys = 30
nzs = 40


In [11]:
nxs

30

In [12]:
# Want to place directly on grid, dx=4
grid_sx_locs = np.arange(xmid-(nxs//2*dx),xmid+(nxs//2*dx), 4)
grid_sy_locs = np.arange(ymid-(nys//2*dx),ymid+(nys//2*dx), 4)
grid_sz_locs = np.arange(zloc-(nzs//2*dx),zloc+(nzs//2*dx), 2)

In [13]:
grid_sx_locs

array([470., 474., 478., 482., 486., 490., 494., 498., 502., 506., 510.,
       514., 518., 522., 526.])

In [14]:
sx, sy, sz = np.meshgrid(grid_sx_locs,
                         grid_sy_locs,
                         grid_sz_locs) 
srclocs = np.vstack((sx.flatten(), sy.flatten(), sz.flatten()))# y is y, I fix the SOFI ordering later
srclocs.shape

(3, 9000)

In [15]:
src_xyz_list = []

tmp_src_xyz_cutoff = 15000 # 4
for srcloc in srclocs[:,:tmp_src_xyz_cutoff].T:
    src_xyz_list.append(srcloc)
# src_xyz_list

In [16]:
# MAKE STRINGS PER SHOT LOC
td = dt  # ignition time
fc = 20  # central frequency
amp = 1.  # amplitude

src_xyz_list=src_xyz_list[::2]
string_list = []
for src in src_xyz_list:
    string_list.append([str(i) for i in [src[0], src[2], src[1], td, fc, amp]])

# WRITE FILES

In [17]:
len(src_xyz_list)

4500

In [18]:
# SELECT GROUPING OF SOURCES
src_per_group = 100

ns = len(src_xyz_list)
num_src_groups = int(ns/src_per_group)

print(num_src_groups)

45


In [19]:
src_cuts = np.arange(0,ns,src_per_group).astype('int')
for i,strt in enumerate(src_cuts):
    groupname = "multishot_group%.2i"%i
    print(groupname)
    
    # Write source file
    group_strings = string_list[strt:strt+src_per_group]
    src_filename = groupname+'.dat'
    # print(src_filename)
    src_file = os.path.join(source_paths,src_filename)  
    # print(src_file)  
    text_file = open(src_file, "wt")
    for string_line in group_strings:
        text_file.writelines('\t'.join(string_line)+'\n')
    text_file.close()

    # Make SOFI Source String
    sofi_src_file = os.path.join(sofi_src_dir,src_filename)
    src_str = get_source_str(sfile=sofi_src_file, multisource=1)
    # print(src_str)
    
    # Make Monitor String
    monitor_strs = get_monitor_str(tsnap_params, 
                               smgrm_ndt, 
                               groupname, 
                               sbsmp_xyz=snap_sbsmp_xyz,
                               snap=False
                              )

    # Write SOFI FILE
    sofi_json_file = os.path.join(shaheen_cp_loc, groupname+'.json')
    shaheen_sofi_json_file = os.path.join(shaheen_proj_loc, groupname+'.json')
    print(sofi_src_file)
    print(sofi_json_file)
    write_SOFIjsonParams(default_strs, 
                         monitor_strs,  
                         t_str, 
                         mod_strs, 
                         src_str, 
                         bndry_str,
                         sofi_json_file)

    # WRITE SLURM FILE
    slurmfile = os.path.join(shaheen_cp_loc, 'submit_'+groupname+'.slurm')
    slurmfilecreation(slurmfile, 
                      gid=i, 
                      sofimaster=sofimaster, 
                      jsonloc=shaheen_sofi_json_file[:-5], 
                      joutloc=shaheen_sofi_json_file[:-5],)
    print(' ')

multishot_group00
/home/birniece/SOFI_UtahFORGE/explosive_multishot/UtahFORGE_GriddedRecs_CroppedModel/inputs/multishot_sources/multishot_group00.dat
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/UtahFORGE_GriddedRecs_CroppedModel/multishot_group00.json
 
multishot_group01
/home/birniece/SOFI_UtahFORGE/explosive_multishot/UtahFORGE_GriddedRecs_CroppedModel/inputs/multishot_sources/multishot_group01.dat
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/UtahFORGE_GriddedRecs_CroppedModel/multishot_group01.json
 
multishot_group02
/home/birniece/SOFI_UtahFORGE/explosive_multishot/UtahFORGE_GriddedRecs_CroppedModel/inputs/multishot_sources/multishot_group02.dat
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/UtahFORGE_GriddedRecs_CroppedModel/multishot_group02.json
 
multishot_group03
/home/birniece/SOFI_UtahFORGE/explosive_multishot/UtahFORGE_GriddedRecs_CroppedModel/inputs/multishot_sour

# MAKE BASH SCRIPT TO SUBMIT ALL SLURM JOBS

In [20]:
slurmfiles = []
for file in glob.glob(shaheen_cp_loc+"/*.slurm"):
    slurmfiles.append(file.split('/')[-1])


In [21]:
slurmfiles[0]

'submit_multishot_group31.slurm'

In [22]:
bashsubmitfile = os.path.join(shaheen_cp_loc, 'submit_ALL.sh')

bashsubmitstring = '''#!/bin/bash 

mkdir -p outputs
mkdir -p outputs/log
mkdir -p outputs/snap
mkdir -p outputs/su

'''

for slurmf in slurmfiles:
    bashsubmitstring+='sbatch %s\n'%slurmf 
print(bashsubmitstring)

text_file = open(bashsubmitfile, "wt")
text_file.writelines(bashsubmitstring)
text_file.close()

#!/bin/bash 

mkdir -p outputs
mkdir -p outputs/log
mkdir -p outputs/snap
mkdir -p outputs/su

sbatch submit_multishot_group31.slurm
sbatch submit_multishot_group08.slurm
sbatch submit_multishot_group37.slurm
sbatch submit_multishot_group21.slurm
sbatch submit_multishot_group33.slurm
sbatch submit_multishot_group19.slurm
sbatch submit_multishot_group10.slurm
sbatch submit_multishot_group00.slurm
sbatch submit_multishot_group20.slurm
sbatch submit_multishot_group16.slurm
sbatch submit_multishot_group34.slurm
sbatch submit_multishot_group25.slurm
sbatch submit_multishot_group23.slurm
sbatch submit_multishot_group09.slurm
sbatch submit_multishot_group43.slurm
sbatch submit_multishot_group27.slurm
sbatch submit_multishot_group12.slurm
sbatch submit_multishot_group39.slurm
sbatch submit_multishot_group41.slurm
sbatch submit_multishot_group40.slurm
sbatch submit_multishot_group38.slurm
sbatch submit_multishot_group24.slurm
sbatch submit_multishot_group35.slurm
sbatch submit_multishot_group22